In [2]:
!pip install pyscipopt

In [3]:
#using SCIP solver
import pyscipopt

def bin_packing_solver(item_sizes, bin_capacity):
    model = pyscipopt.Model("BinPacking")

    #binary variables for item-bin combo
    num_items = len(item_sizes)
    total_weight = sum(item_sizes)
    max_bins = (total_weight // bin_capacity) + 1
    vars = {}

    for i in range(num_items):
        for b in range(max_bins):
            vars[i, b] = model.addVar(vtype="B", name=f"x_{i}_{b}")

    #Objective- to minimize the number of bins used
    bin_used = [model.addVar(vtype="B", name=f"bin_{b}") for b in range(max_bins)]
    model.setObjective(sum(bin_used), "minimize")

    #Each item must be in exactly one bin
    for i in range(num_items):
        model.addCons(sum(vars[i, b] for b in range(max_bins)) == 1)

    #total size of items in each bin cannot exceed bin capacity
    for b in range(max_bins):
        model.addCons(sum(vars[i, b] * item_sizes[i] for i in range(num_items)) <= bin_capacity * bin_used[b])

    model.setParam('limits/time', 600)
    model.setParam('parallel/maxnthreads', 4)
    model.relax()
    model.optimize()

    #solution from SCIP
    if model.getStatus() == "optimal":
        print("Optimal solution found!")
        solution = {}
        for b in range(max_bins):
            if model.getVal(bin_used[b]) > 0.5:#checking if the bin is used
                solution[b] = [i for i in range(num_items) if model.getVal(vars[i, b]) > 0.5]
        return solution
    else:
        print("No optimal solution found.")
        return None

In [4]:
item_sizes = [5, 7, 8, 3, 2]
bin_capacity = 10

result = bin_packing_solver(item_sizes, bin_capacity)
if result:
    for bin_num, items in result.items():
        print(f"Bin {bin_num}: Items {items}")

Optimal solution found!
Bin 0: Items [0, 3]
Bin 1: Items [1, 2]


In [5]:
item_sizes = [5, 8, 2, 5, 9]
bin_capacity = 10

result = bin_packing_solver(item_sizes, bin_capacity)
if result:
    for bin_num, items in result.items():
        print(f"Bin {bin_num}: Items {items}")

Optimal solution found!
Bin 0: Items [4]
Bin 1: Items [0, 1]
Bin 2: Items [2, 3]


In [6]:
#first-fit deacreasing heuristic- offline

def first_fit_dec(n,c,l):
    bin_count = 0
    bins = [0]*n #worst case each item in separate bin
    bin_id=[0]*n
    l.sort(reverse=True)
    print(l)
    for i in range(n):
        curr = 0
        while(curr < bin_count):
            if (bins[curr] >= l[i]):
                bins[curr] = bins[curr]-l[i]
                break
            curr+=1

        if (curr == bin_count):
            bins[bin_count] = c - l[i]
            bin_count+=1
        bin_id[i]=curr

    return bin_count, bin_id

In [6]:
# t=int(input())

# for x in range(t):
#     n,c=map(int,input().split())
#     l=list(map(int,input().split()))
#     sol,bin_res=first_fit_dec(n,c,l)
#     print(sol)
#     print(bin_res)

1
5 10
5 8 2 5 9
[9, 8, 5, 5, 2]
3
[0, 1, 2, 2, 1]


In [7]:
#using heuristic solution from ffd as a start for the lp
def bin_packing_ffd(item_sizes, bin_capacity, ffd_val,ffd_sol):
    model = pyscipopt.Model("BinPacking")

    #binary variables for item-bin combo
    num_items = len(item_sizes)
    total_weight = sum(item_sizes)
    max_bins = (total_weight // bin_capacity) + 1
    vars = {}

    for i in range(num_items):
        for b in range(max_bins):
            vars[i, b] = model.addVar(vtype="B", name=f"x_{i}_{b}")

    #Objective- to minimize the number of bins used
    bin_used = [model.addVar(vtype="B", name=f"bin_{b}") for b in range(max_bins)]
    model.setObjective(sum(bin_used), "minimize")

    #Each item must be in exactly one bin
    for i in range(num_items):
        model.addCons(sum(vars[i, b] for b in range(max_bins)) == 1)

    #total size of items in each bin cannot exceed bin capacity
    for b in range(max_bins):
        model.addCons(sum(vars[i, b] * item_sizes[i] for i in range(num_items)) <= bin_capacity * bin_used[b])

    #using ffd as a start
    if ffd_val and len(ffd_sol) == 2:
        n, item_to_bin = ffd_sol
        if n > 0 and len(i) == num_items:
            for i in range(num_items):
                vars[i, item_to_bin[i]].setInitialValue(1)
            for b in range(n):
                bin_used[b].setInitialValue(1)


    model.setParam('limits/time', 600)
    model.setParam('parallel/maxnthreads', 4)
    model.relax()
    model.optimize()

    #solution from SCIP
    if model.getStatus() == "optimal":
        print("Optimal solution found!")
        solution = {}
        for b in range(max_bins):
            if model.getVal(bin_used[b]) > 0.5:#checking if the bin is used
                solution[b] = [i for i in range(num_items) if model.getVal(vars[i, b]) > 0.5]
        return solution
    else:
        print("No optimal solution found.")
        return None

In [8]:
import os

def read_bpp(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extract number of items and bin capacity from the first two lines
    n = int(lines[0].strip())           # Number of items
    c = int(lines[1].strip())           # Bin capacity

    # Extract item weights from the remaining lines
    weights = [int(line.strip()) for line in lines[2:]]

    if len(weights) != n:
        raise ValueError(f"Mismatch")
    return n, c, weights


def read_all():
    base_dir = '/content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U'
    file_prefix = 'Falkenauer_u250_'
    file_count = 20

    bpp_data = {}

    for i in range(file_count):
        file_name = f"{file_prefix}{str(i).zfill(2)}.txt"
        file_path = os.path.join(base_dir, file_name)

        if os.path.exists(file_path):
            print(f"Reading file: {file_path}")
            n, c, weights = read_bpp(file_path)
            bpp_data[file_name] = {'n': n, 'capacity': c, 'weights': weights}
        else:
            print(f"File not found: {file_path}")

    return bpp_data

In [10]:
data = read_all()

Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_00.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_01.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_02.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_03.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_04.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_05.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_06.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_07.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_08.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_09.txt
Reading file: /content/drive/MyDrive/Falkenauer/Fa

In [11]:
for file_name,file_data in data.items():
  print(f"{file_name}")
  c=file_data['capacity']
  #print(f"Item weights: {file_data['weights']}")
  b=file_data['weights']
  result = bin_packing_solver(b, c)
  if result:
    for bin_num, items in result.items():
        print(f"Bin {bin_num}: Items {items}")
  print()

Falkenauer_u250_00.txt
Optimal solution found!
Bin 0: Items [15]
Bin 1: Items [83]
Bin 2: Items [236]
Bin 3: Items [16, 214]
Bin 4: Items [26, 235]
Bin 5: Items [21]
Bin 6: Items [108]
Bin 7: Items []
Bin 8: Items [24, 109]
Bin 9: Items [72, 104]
Bin 10: Items [144]
Bin 11: Items [222]
Bin 12: Items []
Bin 13: Items [112]
Bin 14: Items []
Bin 15: Items []
Bin 16: Items []
Bin 17: Items [199]
Bin 18: Items [38]
Bin 19: Items [85]
Bin 20: Items [17]
Bin 21: Items [75]
Bin 22: Items [31, 106]
Bin 23: Items []
Bin 24: Items [27, 215]
Bin 25: Items [32]
Bin 26: Items [35, 224]
Bin 27: Items []
Bin 28: Items [14]
Bin 29: Items [28, 237]
Bin 30: Items []
Bin 31: Items [23]
Bin 32: Items [70]
Bin 33: Items [34]
Bin 34: Items []
Bin 35: Items [121]
Bin 36: Items [36]
Bin 37: Items [73]
Bin 38: Items [76, 245]
Bin 39: Items [81, 231]
Bin 40: Items [45, 153]
Bin 41: Items [20]
Bin 42: Items [213]
Bin 43: Items [52]
Bin 44: Items [111]
Bin 45: Items [123]
Bin 46: Items [25]
Bin 47: Items [47]
Bin 

In [12]:
for file_name,file_data in data.items():
  print(f"{file_name}")
  c=file_data['capacity']
  #print(f"Item weights: {file_data['weights']}")
  b=file_data['weights']
  r1,r2 = first_fit_dec(len(b),c,b)
  print(r1)
  # print(r1,r2)
  # result = bin_packing_ffd(b, c, r1, r2)
  # if result:
  #     for bin_num, items in result.items():
  #         print(f"Bin {bin_num}: Items {items}")
  print("\n\n")

Falkenauer_u250_00.txt
[100, 100, 100, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98, 97, 97, 97, 96, 96, 95, 95, 95, 94, 94, 93, 93, 92, 92, 92, 91, 91, 90, 90, 90, 88, 88, 87, 86, 85, 85, 85, 84, 84, 84, 84, 84, 83, 83, 82, 82, 82, 81, 81, 81, 81, 80, 80, 80, 80, 80, 80, 79, 79, 79, 79, 78, 78, 78, 78, 78, 78, 76, 76, 75, 75, 74, 74, 74, 73, 73, 73, 73, 72, 72, 72, 71, 71, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 68, 67, 67, 67, 67, 67, 66, 66, 66, 65, 65, 64, 64, 62, 62, 62, 61, 61, 60, 60, 60, 60, 60, 60, 59, 59, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 55, 55, 55, 55, 55, 53, 53, 53, 53, 53, 53, 52, 52, 50, 50, 49, 49, 49, 49, 49, 48, 48, 47, 47, 47, 47, 46, 46, 46, 46, 45, 45, 45, 45, 45, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 37, 37, 36, 36, 36, 36, 36, 36, 35, 35, 33, 33, 33, 33, 32, 32, 32, 32, 30, 30, 30, 30, 30, 29, 29, 29, 28, 27, 27, 27, 27, 27, 26, 25, 25, 25, 24, 24, 24, 23, 23, 23, 23, 23, 2